# 4. divide and conquer - 分治模式

首先将zerollama目录加入python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入模块

In [2]:
import gevent
from zerollama.agents import AssistantAgent
from zerollama.agents import Session
from zerollama.agents import SummaryAgent
from zerollama.agents import Agent

设定 llm 参数

In [3]:
llm_config = {"model": "Qwen/Qwen2-7B-Instruct-GPTQ-Int4", "type": "openai", "base_url": 'http://localhost:8080/v1/'}

作家Agent可以使用上节定义好的

In [4]:
writer = AssistantAgent(
    name="Writer",
    system_message="你是一名作家。"
                   "您就给定主题撰写引人入胜且简洁的博客文章（带标题）。"
                   "您必须根据收到的反馈完善您的写作并给出完善的版本。"
                   "仅返回您的最终作品，无需附加评论。仅返回您的最终作品，无需附加评论。仅返回您的最终作品，无需附加评论。",
    llm_config=llm_config,
)


工作流也用上节定义好的

In [5]:
def discuss_and_improve(task):
    session = Session(participants=[writer, critic])
    session.append((critic, task))
    session.chat(max_turns=5)
    return session

任务也用上节定义好的

In [6]:
task = '''
    写一篇简洁但引人入胜的关于 DeepLearning.AI 博客文
       DeepLearning.AI. 确保文章在100字以内。
'''

这次我们的评论家Agent包含三个子Agent，分别使用搜索引擎优化审核员、法律审阅者、道德审核员三个角度对文章进行评审，SummaryAgent总结三个审核员的内容，元审核员汇总，内容提出最终建议。

In [7]:
class CriticAgent(Agent):
    def __init__(self, name: str = "Critic"):
        super().__init__(name, None)

        self.SEO_reviewer = AssistantAgent(
            name="SEO Reviewer",
            llm_config=llm_config,
            system_message="您是一名 SEO 审核员，以优化搜索引擎内容的能力而闻名，"
                           "确保其排名良好并吸引自然流量。确保您的建议简洁（3 个要点以内）、具体且切题。"
                           "通过陈述您的角色来开始审查。",
        )

        self.legal_reviewer = AssistantAgent(
            name="Legal Reviewer",
            llm_config=llm_config,
            system_message="您是一名法律审核员，以确保内容合法且不存在任何潜在法律问题的能力而闻名。"
                           "确保您的建议简洁（3 个要点以内）、具体且切中要点。"
                           "通过陈述您的角色来开始审查。",
        )

        self.ethics_reviewer = AssistantAgent(
            name="Ethics Reviewer",
            llm_config=llm_config,
            system_message="您是一名道德审核员，以确保内容符合道德规范且不存在任何潜在道德问题的能力而闻名。"
                           "确保您的建议简洁（3 个要点以内）、具体且切中要点。"
                           "通过陈述您的角色来开始审查。",
        )

        self.summary_agent = SummaryAgent(
            system_message="Return review into as JSON object only:"
                           "{'审稿人': '', '评论': ''}.",
            llm_config=llm_config)

        self.meta_reviewer = AssistantAgent(
            name="Meta Reviewer",
            llm_config=llm_config,
            system_message="您是元审核员，您汇总和审阅其他审核员的工作，并对内容提出最终建议。",
        )

        self.critices = [self.SEO_reviewer, self.legal_reviewer, self.ethics_reviewer]

    def generate_reply(self, messages, stream=False, options=None):
        task = messages[0]["content"]
        article = messages[-1]["content"]

        # divide
        def review_and_summary(critic, article):
            session = Session(participants=[self.summary_agent, critic])
            session.append((self.summary_agent, article))
            session.chat(max_turns=1, verbose=False, verbose_history=False)

            summary = self.summary_agent.summary(session)
            return summary

        review = gevent.joinall([gevent.spawn(review_and_summary, critic, article) for critic in self.critices])
        review = "\n".join([x.value for x in review])

        # conquer
        session2 = Session(participants=[self.summary_agent, self.meta_reviewer])
        session2.append((self.summary_agent, review))
        session2.chat(max_turns=1)

        return task + "\n\n" + session2.history[-1].content

使用gevent并发加快速度

In [8]:
critic = CriticAgent(name="Critic")

看看效果吧

In [9]:
session = discuss_and_improve(task)

Critic (round 0)


    写一篇简洁但引人入胜的关于 DeepLearning.AI 博客文
       DeepLearning.AI. 确保文章在100字以内。


Writer (round 0)

🌟🚀探索AI的新纪元🌟🚀：欢迎来到DeepLearning.AI

🎉揭秘未来科技的核心！🎉 DeepLearning.AI带你深入机器学习、深度学习及人工智能的奇妙世界。这里的每一篇文章都是对你思想的刺激与启发。

🛠️动手实操，掌握前沿技术！🛠️从理论到实践，我们的教程和案例研究将带领你构建高效算法，解决现实生活中的复杂问题。

💡深度解析，洞察AI奥秘！💡学习最新研究动态、AI伦理讨论，让你站在技术变革的前沿。

🚀即刻起航，开启你的AI之旅🚀，加入DeepLearning.AI的大家庭，一起探索这个充满无限可能的新世界。

快来加入我们吧！🚀🎯一起创造未来，让科技为人类带来更美好的明天！

SummaryAgent (round 0)

{
  '审稿人': 'Legal Reviewer',
  '评论': '注意内容合规性，确保用户数据安全、版权合法使用及内容准确性，明确版权声明并谨慎处理第三方内容及实践案例'
}
{"审稿人": "Ethics Reviewer", "评论": "在讲解AI和深度学习的过程中，强调伦理透明性、促进多样性和包容性，关注AI的伦理影响和正面社会贡献。"}
{
  "审稿人": "SEO Reviewer",
  "评论": "对于这个内容，可以从以下几个方面进行优化：首先是关键词优化，可以通过增加与内容主题相关的高频关键词（如'人工智能'、'机器学习'、'深度学习'等）和长尾关键词（如'AI在医疗领域的应用'、'深度学习入门指南'），来优化其搜索引擎表现并吸引流量。其次是内部链接结构，通过在文章中嵌入链接到DeepLearning.AI的其他相关内容，可以提升用户体验，加深用户对网站内容的了解，同时也有助于搜索引擎对网站的整体结构和内容质量的评估。最后是外部链接建设，通过引用和链接到外部的权威资源，可以增加网站的可信度和专家级别，通过这些链接能够吸引流量，并且它们也是搜索引擎中考虑的排名因素之一，有助于提高网站的排名。"
}

Meta Reviewer (round 0)

综合多位审稿人的意见，

## 总结
divide and conquer - 分治模式，将一个大任务分成多个子任务，让大语言模型扮演多个角色分别完成子任务，汇总形成更全面的结果